In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pretty_midi
import visual_midi
import joblib
from os import listdir
from os.path import getsize
from MIDIComposingAI.utils import piano_roll_to_pretty_midi
from MIDIComposingAI.create_dataset import *
from MIDIComposingAI.get_back_data import *
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.preprocessing import Normalizer, MinMaxScaler

### Get data

In [3]:
path = '../raw_data/pretty_midi'
directory = [file_name for file_name in listdir(path) if getsize(f'{path}/{file_name}') < 300_000]

for i, file_name in enumerate(directory):
    file = joblib.load(f'{path}/{file_name}')
    if i == 0:
        X, y = create_dataset(file, ratio=0.2, store=False)
    else:
        try:
            loaded = create_dataset(file, ratio=0.2, store=False)
            X = np.concatenate((X, loaded[0]))
            y = np.concatenate((y, loaded[1]))
            del([loaded, file])
        except:
            pass
    if i % 10 == 0:
        print(f'{X.shape[0]} observations')
    if X.shape[0] >= 1_500:
        break

chord = adding_chords_info('../raw_data/chords_midi.csv', X)

X = X.reshape((X.shape[0], -1))
X = np.concatenate((chord, X), axis=1, dtype=np.int8)

y = y.reshape((y.shape[0], -1))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=2)



14 observations
95 observations
240 observations
369 observations
505 observations
585 observations
690 observations
774 observations
918 observations
987 observations
1088 observations
1224 observations
1347 observations
1481 observations


### Make predictions

#### Initiate, train and save our model

In [4]:
scaler = MinMaxScaler().fit(X_train)
X_train = scaler.transform(X_train)

In [5]:
tree = DecisionTreeRegressor()
tree.fit(X_train, y_train)
joblib.dump(tree, '../Models/normalized_tree.joblib')
# tree = joblib.load('../Models/tree.joblib')

['../Models/normalized_tree.joblib']

#### Attempt to debug it

In [6]:
# example_file = []
# for file in directory:
#     if 200_000 > getsize(f'{path}/{file}'): # We don't want too big or too little files
#         example_file.append(joblib.load(f'{path}/{file}'))
#         break

In [7]:
# dir = 'first_pres_dataset'
# for i, file in enumerate(example_file):
#     create_nparray_dataset(file, dir, name=f'chose{i}')

In [8]:
# X_test, y_test = joblib.load(f'../raw_data/pandas_dataframes/first_pres_dataset/chose0')

In [9]:
# X_test, y_test = create_nparray_dataset(example_file[0], 'f', 'n', store=False)

In [10]:
# X_test = X_test.reshape((X_test.shape[0], -1))
# y_test = y_test.reshape((y_test.shape[0], -1))

#### True thing

In [11]:
# Making our predictions
X_test_scale = scaler.transform(X_test)
predictions = tree.predict(X_test_scale)

In [12]:
# Test for the "false predictions" problem
for i, pred in enumerate(predictions):
    if i % 10 == 0:
        print((pred == y_test[i]).mean())

0.009
0.545


In [13]:
predicted_melodies = np.array([assembled_target_to_melody(prediction) for prediction in predictions])
true_melodies = np.array([assembled_target_to_melody(melody) for melody in y_test])

In [14]:
# They should be different

In [15]:
X_test = np.delete(X_test, np.s_[0:24], axis=1)
X_test = X_test.reshape(-1, 128, 500)

In [16]:
%%capture --no-display

coloring = visual_midi.presets.Coloring.INSTRUMENT
plotter = visual_midi.Plotter(coloring=coloring)
plotter.show_notebook(piano_roll_to_pretty_midi(X_test[4]))

Loading BokehJS ...

Column(id='1114', ...)

In [17]:
%%capture --no-display

coloring = visual_midi.presets.Coloring.INSTRUMENT
plotter = visual_midi.Plotter(coloring=coloring)
plotter.show_notebook(piano_roll_to_pretty_midi(predicted_melodies[4]))

Loading BokehJS ...

Column(id='1239', ...)

In [18]:
%%capture --no-display

coloring = visual_midi.presets.Coloring.INSTRUMENT
plotter = visual_midi.Plotter(coloring=coloring)
plotter.show_notebook(piano_roll_to_pretty_midi(true_melodies[4]))

Loading BokehJS ...

Column(id='1389', ...)

In [19]:
copy = X_test.copy()
copy_2 = X_test.copy()

In [20]:
assembled_predicted_music = np.array(
    [assemblate_accompaniment_melody(test, predicted_melody) for test, predicted_melody in zip(copy, predicted_melodies)],
)

assembled_true_music = np.array(
    [assemblate_accompaniment_melody(test, true_melody) for test, true_melody in zip(copy_2, true_melodies)],
)

In [22]:
for i, (pred, true) in enumerate(zip(assembled_predicted_music, assembled_true_music)):
    try:
        piano_roll_to_pretty_midi(pred, fs=50).write(f'../PredVSTrue/MinMaxScaledRegressor/{i}_pred.mid')
        piano_roll_to_pretty_midi(true, fs=50).write(f'../PredVSTrue/MinMaxScaledRegressor/{i}_true.mid')
    except:
        pass